<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%AE%E3%83%A3%E3%83%A9%E3%83%AA%E3%83%BC%E3%81%AE%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y jps_gallery_tools

Found existing installation: jps-gallery-tools 0.0.1
Uninstalling jps-gallery-tools-0.0.1:
  Successfully uninstalled jps-gallery-tools-0.0.1


In [2]:
!pip install --upgrade git+https://github.com/nakamura196/jps_gallery_tools.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nakamura196/jps_gallery_tools.git to /tmp/pip-req-build-4qa5wml1
  Running command git clone -q https://github.com/nakamura196/jps_gallery_tools.git /tmp/pip-req-build-4qa5wml1
  Created wheel for jps-gallery-tools: filename=jps_gallery_tools-0.0.1-py3-none-any.whl size=9866 sha256=2e811f57439d7852079877b0480518bf640411ee0d931fde9753e34fe622c81c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5hx1c2a3/wheels/65/83/8e/8c01de00316cafa2af0853c4606f20a3b6071ee23434b7463b
Successfully built jps-gallery-tools


In [3]:
from jps_gallery_tools.api import UrlImage, Part, JPSGalleryClient, IIIF

In [4]:
import json
import hashlib
import os
import requests

file = "tasks/curation/curation.json"

if not os.path.exists(file):
  df = requests.get("https://utda.github.io/tenjiroom/genelib_vm2020-01.json").json()
  os.makedirs(os.path.dirname(file), exist_ok=True)
  with open(file, 'w') as f:
    json.dump(df, f, indent=2, ensure_ascii=False)

with open(file) as f:
    df = json.load(f)

In [5]:
gallery = JPSGalleryClient()

label = df["label"]

gallery.setMetadata("title", "ja", label)

In [6]:
values = df["metadata"][0]["value"]
map = {}

for value in values:
    map[value] = {}

from pprint import pprint

pprint(map)

{'明治時代の東大を見る\u3000－写真帖『東京帝國大學』から－': {},
 '絵巻物を比較して見る\u3000－百鬼夜行図から－': {},
 '総合図書館資料アラカルト': {}}


In [7]:
selections = df["selections"]

for selection in selections:
    manifest = selection["within"]["@id"]

    hs = hashlib.md5(manifest.encode()).hexdigest()

    manifest_path = f"tasks/curation/manifests/{hs}.json"

    if not os.path.exists(manifest_path):
        os.makedirs(os.path.dirname(manifest_path), exist_ok=True)

        m = requests.get(manifest).json()

        with open(manifest_path, "w") as f:
            json.dump(m, f)

    m = json.load(open(manifest_path))

    canvas_image_map = {}

    for canvas in m["sequences"][0]["canvases"]:
        canvas_image_map[canvas["@id"]] = canvas["images"][0]["resource"]["service"]["@id"]

    members = selection["members"]

    for member in members:

        member_id = member["@id"]

        spl = member_id.split("#xywh=")

        canvas_id = spl[0]

        xywh = None

        if len(spl) > 1:
            xywh = spl[1] # .split(",")

            print(canvas_id, xywh)

        room = member["metadata"][0]["value"]

        room_label = values[room["roomId"]]

        item = {
            # "member_id": member_id,
            "manifest": manifest,
            "label": member["label"],
            "infoJSonUrl": canvas_image_map[canvas_id] + "/info.json",
        }

        map[room_label][room["position"]] = item

        if xywh is not None:
            # item["x"] = float(xywh[0])
            # item["y"] = float(xywh[1])
            # item["w"] = float(xywh[2])
            # item["h"] = float(xywh[3])
            item["xywh"] = xywh

        # break

    
    # break

pprint(map)

https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p37 114,252,2740,1220
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p42 430,300,1900,1492
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p45 426,300,1892,1484
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p49 406,312,1920,1500
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p53 414,304,1924,1488
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p57 418,324,1916,1464
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p61 438,312,1900,1500
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p67 442,304,1900,1496
https://iiif.dl.itc.u-tokyo.ac.jp/repo/iiif/8b0089a9-66fc-4833-ab13-21d92f7f7aa4/canvas/p77 418,304,1956,1512
https://ii

In [8]:
from jps_gallery_tools.api import ListPart, ListExternalLinkPart
for room_label in values:
    part3 = ListPart("tile")
    part3.setMetadata("title", "ja", room_label)

    items = map[room_label]

    for pos in sorted(items):
        item = items[pos]

        part32 = ListExternalLinkPart()
        # part32.setUrl(f"https://cultural.jp/item/{manifest['id']}")
        part32.setMetadata("title", "ja", item["label"])
        # part32.setMetadata("title", "en", manifest_label)

        xywh = item["xywh"] if "xywh" in item else None

        image = IIIF(item["manifest"], item["infoJSonUrl"], item["label"], "", xywh=xywh) # , xywh=xywh
        part32.setImage(image)

        part3.addPart(part32)

    gallery.addPart(part3)

In [9]:
data = gallery.export()

with open(f"{file}_mod.json", 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

arr = []

arr.append(json.dumps(data, ensure_ascii=False))

with open(f"{file}_mod.jsonl", mode='w') as f:
    f.write("\n".join(arr))